In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# df_short = pd.read_json('/allah/freqtrade/json_dict/2023-09-06_17-41-16_MacdStrategyShort.json')
df_long = pd.read_json('/allah/freqtrade/json_dict/2023-09-06_18-00-38_MacdStrategyLong.json')


In [2]:
""" 30 Volume Version """
import numpy as np

lag_features = ['volume', 'close']  # List of features to create lag columns for
lag_columns = []  # List to store the names of generated lag columns

for feature in lag_features:
    for i in range(1, 31):
        df_long[f'{feature}_{i}'] = df_long[feature].shift(i)
        # Append the generated lag column names to the lag_columns list
        lag_columns.append(f'{feature}_{i}')

# Now, lag_columns contains the names of all generated lag columns
df_long['macd_direction'] = np.where(df_long.macdhist > 0, 1, -1)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from scipy.special import expit  # Sigmoid function

# Assuming you have already prepared your dataset, features, and target variable.
# Replace df_long, lag_columns, and trade with your actual dataset.

# Step 1: Select features and target variable
features = lag_columns 
target = 'label'

# Assuming df_long is your dataset
label_encoder = LabelEncoder()
df_long['label'] = label_encoder.fit_transform(df_long['label'])
train_df = df_long[df_long['trade'] == 1.0]

# Step 2: Split the data into training and testing sets
X = train_df[features].values
y = train_df[target].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Scale the features (MinMaxScaler)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Define the neural network model parameters
input_size = len(features)
hidden_size = 128
dropout_prob = 0.2
output_size = 1

# Initialize weights and biases
weights_input_hidden = np.random.randn(input_size, hidden_size)
biases_input_hidden = np.zeros((1, hidden_size))
weights_hidden_output = np.random.randn(hidden_size, output_size)
biases_hidden_output = np.zeros((1, output_size))

# Define the sigmoid activation function
def sigmoid(x):
    return expit(x)

# Define the derivative of the sigmoid function
def sigmoid_derivative(x):
    return x * (1 - x)

# Define the learning rate and number of epochs
learning_rate = 0.0001
num_epochs = 5000

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    input_layer = X_train_scaled
    hidden_layer_input = np.dot(input_layer, weights_input_hidden) + biases_input_hidden
    hidden_layer_output = sigmoid(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + biases_hidden_output
    output_layer_output = sigmoid(output_layer_input)

    # Calculate loss
    loss = -np.mean(y_train * np.log(output_layer_output) + (1 - y_train) * np.log(1 - output_layer_output))

    # Backpropagation
    d_loss = output_layer_output - y_train
    d_output_layer = d_loss * sigmoid_derivative(output_layer_output)
    d_hidden_layer = np.dot(d_output_layer, weights_hidden_output.T) * sigmoid_derivative(hidden_layer_output)

    # Update weights and biases
    weights_hidden_output -= learning_rate * np.dot(hidden_layer_output.T, d_output_layer)
    biases_hidden_output -= learning_rate * np.sum(d_output_layer, axis=0)
    weights_input_hidden -= learning_rate * np.dot(input_layer.T, d_hidden_layer)
    biases_input_hidden -= learning_rate * np.sum(d_hidden_layer, axis=0)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss = {loss:.4f}")

# Step 5: Evaluate the model
input_layer_test = X_test_scaled
hidden_layer_input_test = np.dot(input_layer_test, weights_input_hidden) + biases_input_hidden
hidden_layer_output_test = sigmoid(hidden_layer_input_test)
output_layer_input_test = np.dot(hidden_layer_output_test, weights_hidden_output) + biases_hidden_output
output_layer_output_test = sigmoid(output_layer_input_test)

y_pred = (output_layer_output_test > 0.5).astype(int)
test_accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {test_accuracy:.4f}')


In [23]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard

# Assuming you have already loaded your data

# Step 1: Select features and target variable
features = lag_columns
target = 'label'

# Encode the labels using LabelEncoder
label_encoder = LabelEncoder()
df_long['label'] = label_encoder.fit_transform(df_long['label'])

train_df = df_long[df_long.trade == 1.0]

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    train_df[features], train_df[target], test_size=0.2, random_state=42
)

# Step 3: Scale the features (MinMaxScaler)
scaler = MinMaxScaler()
X_train_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.transform(X_test)

# Count the number of samples in each class
print("Class distribution before oversampling:", Counter(y_train))

# Apply SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Count the number of samples in each class after oversampling
print("Class distribution after oversampling:", Counter(y_train_resampled))

# Step 4: Define the neural network model
model = Sequential([
    Dense(128, input_dim=len(features), activation='relu'),
    Dropout(0.2),  # Adding dropout for regularization
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification, so use 'sigmoid' activation
])

# Specify a higher learning rate for the Adam optimizer
learning_rate = 0.0001  # You can adjust this value
optimizer = Adam(learning_rate=learning_rate)

# Compile the model with the specified optimizer
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Step 5: Train the model with TensorBoard
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)
model.fit(X_train_resampled, y_train_resampled, epochs=5000, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[tensorboard])

# Step 6: Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

# Inverse transform the encoded labels to get the original labels
original_labels = label_encoder.inverse_transform(model.predict(X_test_scaled) > 0.5)


Class distribution before oversampling: Counter({1: 31886, 2: 11251})
Class distribution after oversampling: Counter({1: 31886, 2: 31886})
Epoch 1/5000
1993/1993 [==============================] - 4s 2ms/step - loss: -322.1042 - accuracy: 0.5000 - val_loss: -592.9045 - val_accuracy: 0.7426
Epoch 2/5000
1993/1993 [==============================] - 3s 2ms/step - loss: -3560.2622 - accuracy: 0.5000 - val_loss: -3439.3911 - val_accuracy: 0.7426
Epoch 3/5000
1993/1993 [==============================] - 3s 1ms/step - loss: -12125.4668 - accuracy: 0.5000 - val_loss: -9337.0977 - val_accuracy: 0.7426
Epoch 4/5000
 845/1993 [===========>..................] - ETA: 1s - loss: -22051.1855 - accuracy: 0.5015

KeyboardInterrupt: 

In [21]:
y_train_resampled.value_counts()

label
false_long    31886
true_long     31886
Name: count, dtype: int64